<a href="https://colab.research.google.com/github/emiliamusso/pyspark/blob/main/API_Estructurada_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark -q

# Descarguemos la data con la que trabajaremos
Ejecuta la celda nada mas

In [ ]:
from zipfile import ZipFile
import requests

with open("./credit_cards_data.zip", "wb") as f:
    f.write(requests.get("https://github.com/engcarlosperezmolero/resources_and_tools/blob/main/data/csv/credit_cards_data.zip?raw=true").content)

zip_ref = ZipFile(f"/content/credit_cards_data.zip")
zip_ref.extractall()
zip_ref.close()

### application_record.csv
|Columna| Explicacion|
|-------|------------|
|ID |	Client number	|
|CODE_GENDER |	Gender	|
|FLAG_OWN_CAR |	Is there a car	|
|FLAG_OWN_REALTY |	Is there a property	|
|CNT_CHILDREN |	Number of children	|
|AMT_INCOME_TOTAL |	Annual income	|
|NAME_INCOME_TYPE |	Income category	|
|NAME_EDUCATION_TYPE |	Education level	|
|NAME_FAMILY_STATUS |	Marital status	|
|NAME_HOUSING_TYPE |	Way of living	|
|DAYS_BIRTH |	Birthday	Count backwards from current day (0), -1 means yesterday|
|DAYS_EMPLOYED |	Start date of employment	Count backwards from current day(0). If positive, it means the person currently unemployed. |
|FLAG_MOBIL |	Is there a mobile phone	|
|FLAG_WORK_PHONE |	Is there a work phone	|
|FLAG_PHONE |	Is there a phone	|
|FLAG_EMAIL |	Is there an email	|
|OCCUPATION_TYPE |	Occupation	|
|CNT_FAM_MEMBERS |	Family size	|



### credit_record.csv
|Columna|Explicacion|
|-------|------------|
|ID     |	Client number	|
|MONTHS_BALANCE|Record month The month of the extracted data is the starting point, backwards, 0 is the current month, -1 is the previous month, and so on|
|STATUS|	Status	0: 1-29 days past due 1: 30-59 days past due 2: 60-89 days overdue 3: 90-119 days overdue 4: 120-149 days overdue 5: Overdue or bad debts, write-offs for more than 150 days C: paid off that month X: No loan for the month|
|TOY_STATUS|3 estados de juguete que nos permitiran practicar. Los valores son: good, bad, regular|

### Rutas a los archivos
```'/content/credit_cards_data/application_record.csv'```

```'/content/credit_cards_data/credit_records_toy.csv'```


# Cree la sesion de Spark donde la aplicacion se llame como usted desee.

In [ ]:
import pyspark
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder\
        .master("local[*]")\
        .appName("Ejercicio Api Estructurada Humai")\
        .getOrCreate()

# Lee los archivos dejando que spark infiera los tipos de las columnas. Como se ven los datos?

In [ ]:
app_record = spark.read.csv(pyspark.SparkFiles.get('/content/credit_cards_data/application_record.csv'), header=True, inferSchema=True)
credit_df = spark.read.csv('/content/credit_cards_data/credit_records_toy.csv', header=True, inferSchema=True)

In [ ]:
app_record.show(5, truncate=False)

+-------+-----------+------------+---------------+------------+----------------+--------------------+-----------------------------+--------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+
|ID     |CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|CNT_CHILDREN|AMT_INCOME_TOTAL|NAME_INCOME_TYPE    |NAME_EDUCATION_TYPE          |NAME_FAMILY_STATUS  |NAME_HOUSING_TYPE|DAYS_BIRTH|DAYS_EMPLOYED|FLAG_MOBIL|FLAG_WORK_PHONE|FLAG_PHONE|FLAG_EMAIL|OCCUPATION_TYPE|CNT_FAM_MEMBERS|
+-------+-----------+------------+---------------+------------+----------------+--------------------+-----------------------------+--------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+
|5008804|M          |Y           |Y              |0           |427500.0        |Working             |Higher education             |Civil marriage      |Rented apartment 

# Como se ve el esquema?

In [ ]:
app_record.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- CODE_GENDER: string (nullable = true)
 |-- FLAG_OWN_CAR: string (nullable = true)
 |-- FLAG_OWN_REALTY: string (nullable = true)
 |-- CNT_CHILDREN: integer (nullable = true)
 |-- AMT_INCOME_TOTAL: double (nullable = true)
 |-- NAME_INCOME_TYPE: string (nullable = true)
 |-- NAME_EDUCATION_TYPE: string (nullable = true)
 |-- NAME_FAMILY_STATUS: string (nullable = true)
 |-- NAME_HOUSING_TYPE: string (nullable = true)
 |-- DAYS_BIRTH: integer (nullable = true)
 |-- DAYS_EMPLOYED: integer (nullable = true)
 |-- FLAG_MOBIL: integer (nullable = true)
 |-- FLAG_WORK_PHONE: integer (nullable = true)
 |-- FLAG_PHONE: integer (nullable = true)
 |-- FLAG_EMAIL: integer (nullable = true)
 |-- OCCUPATION_TYPE: string (nullable = true)
 |-- CNT_FAM_MEMBERS: double (nullable = true)



# Selecciona las primeras 3 columnas

In [ ]:
columnas = app_record.columns[:3]

In [ ]:
# usa el atributo .columns y slicing de listas
nuevo_app_record = app_record.select(columnas)

In [ ]:
#muestralas con un select
nuevo_app_record.show()

+-------+-----------+------------+
|     ID|CODE_GENDER|FLAG_OWN_CAR|
+-------+-----------+------------+
|5008804|          M|           Y|
|5008805|          M|           Y|
|5008806|          M|           Y|
|5008808|          F|           N|
|5008809|          F|           N|
|5008810|          F|           N|
|5008811|          F|           N|
|5008812|          F|           N|
|5008813|          F|           N|
|5008814|          F|           N|
|5008815|          M|           Y|
|5112956|          M|           Y|
|6153651|          M|           Y|
|5008819|          M|           Y|
|5008820|          M|           Y|
|5008821|          M|           Y|
|5008822|          M|           Y|
|5008823|          M|           Y|
|5008824|          M|           Y|
|5008825|          F|           Y|
+-------+-----------+------------+
only showing top 20 rows



# Muestra un resumen estadistico de los datos numericos que podrian ser utiles

In [ ]:
#mira la función summary
app_record.select('AMT_INCOME_TOTAL').summary().show(truncate=False)

+-------+------------------+
|summary|AMT_INCOME_TOTAL  |
+-------+------------------+
|count  |438557            |
|mean   |187524.2860095039 |
|stddev |110086.85306622987|
|min    |26100.0           |
|25%    |121500.0          |
|50%    |160650.0          |
|75%    |225000.0          |
|max    |6750000.0         |
+-------+------------------+



In [ ]:
#mira la función describe
app_record.select('AMT_INCOME_TOTAL').describe().show(truncate=False)

+-------+------------------+
|summary|AMT_INCOME_TOTAL  |
+-------+------------------+
|count  |438557            |
|mean   |187524.2860095039 |
|stddev |110086.85306622987|
|min    |26100.0           |
|max    |6750000.0         |
+-------+------------------+



# Formatea los numeros del anterior resultado usando las funciones de spark.sql que hay en F.

nota: puedes usar ```dir(F)``` para tener una idea de que podria estar alli

In [ ]:
import pyspark.sql.functions as F

In [ ]:
dir(F)

['Any',
 'ArrayType',
 'Callable',
 'Column',
 'DataFrame',
 'DataType',
 'Dict',
 'Iterable',
 'JVMView',
 'List',
 'Optional',
 'PandasUDFType',
 'PySparkTypeError',
 'PySparkValueError',
 'SparkContext',
 'StringType',
 'StructType',
 'TYPE_CHECKING',
 'Tuple',
 'Type',
 'Union',
 'UserDefinedFunction',
 'UserDefinedTableFunction',
 'ValuesView',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_create_column_from_literal',
 '_create_lambda',
 '_create_py_udf',
 '_create_py_udtf',
 '_from_numpy_type',
 '_get_jvm_function',
 '_get_lambda_parameters',
 '_invoke_binary_math_function',
 '_invoke_function',
 '_invoke_function_over_columns',
 '_invoke_function_over_seq_of_columns',
 '_invoke_higher_order_function',
 '_options_to_str',
 '_test',
 '_to_java_column',
 '_to_seq',
 '_unresolved_named_lambda_variable',
 'abs',
 'acos',
 'acosh',
 'add_months',
 'aes_decrypt',
 'aes_encrypt',
 'aggregate',
 'any_value',
 'approxC

In [ ]:
# de pyspark.sql.functions hay una function llamada format_number trata de usarla

In [ ]:
from pyspark.sql.functions import format_number, col

In [ ]:
#Resolucion:
app_record.select('AMT_INCOME_TOTAL')\
          .describe()\
          .select(
              "summary",
              format_number(col("AMT_INCOME_TOTAL").cast("Float"), 2).alias("income_format")
          )\
          .show(10, False)


+-------+-------------+
|summary|income_format|
+-------+-------------+
|count  |438,557.00   |
|mean   |187,524.28   |
|stddev |110,086.85   |
|min    |26,100.00    |
|max    |6,750,000.00 |
+-------+-------------+



# realiza una funcion que reciba un dataframe y retorne un dataframe donde la misma realice conteo de missing values para cada columna del dataframe de entrada (nulos y nan)

In [ ]:
# podrias necesitar las siguientes funciones count, isnan, isnull, when, col y usar comprehension de listas dentro del .select()

In [ ]:
from pyspark.sql.functions import count, isnan, isnull, when, col

In [ ]:
app_record.columns

['ID',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'FLAG_MOBIL',
 'FLAG_WORK_PHONE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'OCCUPATION_TYPE',
 'CNT_FAM_MEMBERS']

In [ ]:
app_record.select([count(when(isnull(column), column)).alias(column) for column in app_record.columns]).show(10, False)

+---+-----------+------------+---------------+------------+----------------+----------------+-------------------+------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+
|ID |CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|CNT_CHILDREN|AMT_INCOME_TOTAL|NAME_INCOME_TYPE|NAME_EDUCATION_TYPE|NAME_FAMILY_STATUS|NAME_HOUSING_TYPE|DAYS_BIRTH|DAYS_EMPLOYED|FLAG_MOBIL|FLAG_WORK_PHONE|FLAG_PHONE|FLAG_EMAIL|OCCUPATION_TYPE|CNT_FAM_MEMBERS|
+---+-----------+------------+---------------+------------+----------------+----------------+-------------------+------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+
|0  |0          |0           |0              |0           |0               |0               |0                  |0                 |0                |0         |0            |0         |0              |0         |0         |13420

In [ ]:
app_record.select([count(when(isnan(column) | isnull(column), column)).alias(column) for column in app_record.columns])

DataFrame[ID: bigint, CODE_GENDER: bigint, FLAG_OWN_CAR: bigint, FLAG_OWN_REALTY: bigint, CNT_CHILDREN: bigint, AMT_INCOME_TOTAL: bigint, NAME_INCOME_TYPE: bigint, NAME_EDUCATION_TYPE: bigint, NAME_FAMILY_STATUS: bigint, NAME_HOUSING_TYPE: bigint, DAYS_BIRTH: bigint, DAYS_EMPLOYED: bigint, FLAG_MOBIL: bigint, FLAG_WORK_PHONE: bigint, FLAG_PHONE: bigint, FLAG_EMAIL: bigint, OCCUPATION_TYPE: bigint, CNT_FAM_MEMBERS: bigint]

In [ ]:
# null -> None
# nan -> float

In [ ]:
def count_nan_and_nulls(spark_df):
    return spark_df.select([count(when(isnan(column) | isnull(column), column)).alias(column) for column in spark_df.columns]).toPandas().rename({0: 'missinig_value_counts'}).T

In [ ]:
count_nan_and_nulls(app_record)

,missinig_value_counts
ID,0
CODE_GENDER,0
FLAG_OWN_CAR,0
FLAG_OWN_REALTY,0
CNT_CHILDREN,0
AMT_INCOME_TOTAL,0
NAME_INCOME_TYPE,0
NAME_EDUCATION_TYPE,0
NAME_FAMILY_STATUS,0
NAME_HOUSING_TYPE,0


# consigue un dataframe con solo las columnas de tipo numerico

In [ ]:
# existe un atributo llamado dtypes, explora que arroja y como podrias usar comprehension de listas para filtrar los tipos de las columnas dentro del .select()

In [ ]:
app_record.dtypes

[('ID', 'int'),
 ('CODE_GENDER', 'string'),
 ('FLAG_OWN_CAR', 'string'),
 ('FLAG_OWN_REALTY', 'string'),
 ('CNT_CHILDREN', 'int'),
 ('AMT_INCOME_TOTAL', 'double'),
 ('NAME_INCOME_TYPE', 'string'),
 ('NAME_EDUCATION_TYPE', 'string'),
 ('NAME_FAMILY_STATUS', 'string'),
 ('NAME_HOUSING_TYPE', 'string'),
 ('DAYS_BIRTH', 'int'),
 ('DAYS_EMPLOYED', 'int'),
 ('FLAG_MOBIL', 'int'),
 ('FLAG_WORK_PHONE', 'int'),
 ('FLAG_PHONE', 'int'),
 ('FLAG_EMAIL', 'int'),
 ('OCCUPATION_TYPE', 'string'),
 ('CNT_FAM_MEMBERS', 'double')]

In [ ]:
#Solucion
app_record.select([column for column, dtype in app_record.dtypes if dtype != "string"]).show()

+-------+------------+----------------+----------+-------------+----------+---------------+----------+----------+---------------+
|     ID|CNT_CHILDREN|AMT_INCOME_TOTAL|DAYS_BIRTH|DAYS_EMPLOYED|FLAG_MOBIL|FLAG_WORK_PHONE|FLAG_PHONE|FLAG_EMAIL|CNT_FAM_MEMBERS|
+-------+------------+----------------+----------+-------------+----------+---------------+----------+----------+---------------+
|5008804|           0|        427500.0|    -12005|        -4542|         1|              1|         0|         0|            2.0|
|5008805|           0|        427500.0|    -12005|        -4542|         1|              1|         0|         0|            2.0|
|5008806|           0|        112500.0|    -21474|        -1134|         1|              0|         0|         0|            2.0|
|5008808|           0|        270000.0|    -19110|        -3051|         1|              0|         1|         1|            1.0|
|5008809|           0|        270000.0|    -19110|        -3051|         1|              0

# consigue un dataframe con solo las columnas con texto

In [ ]:
#Solucion parecida a la anterior
app_record.select([column for column, dtype in app_record.dtypes if dtype == "string"]).show()

+-----------+------------+---------------+--------------------+--------------------+--------------------+-----------------+---------------+
|CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|    NAME_INCOME_TYPE| NAME_EDUCATION_TYPE|  NAME_FAMILY_STATUS|NAME_HOUSING_TYPE|OCCUPATION_TYPE|
+-----------+------------+---------------+--------------------+--------------------+--------------------+-----------------+---------------+
|          M|           Y|              Y|             Working|    Higher education|      Civil marriage| Rented apartment|           NULL|
|          M|           Y|              Y|             Working|    Higher education|      Civil marriage| Rented apartment|           NULL|
|          M|           Y|              Y|             Working|Secondary / secon...|             Married|House / apartment| Security staff|
|          F|           N|              Y|Commercial associate|Secondary / secon...|Single / not married|House / apartment|    Sales staff|
|          F|       

# realiza un conteo de los distintos tipos de ingresos que existen

In [ ]:
#Investiga la funcion distinct() para obtener los distintos tipos de ingresos
app_record.select('NAME_INCOME_TYPE').distinct().show()

+--------------------+
|    NAME_INCOME_TYPE|
+--------------------+
|             Student|
|Commercial associate|
|       State servant|
|             Working|
|           Pensioner|
+--------------------+



In [ ]:
#Usa groupby , count y sort
app_record.groupBy('NAME_INCOME_TYPE').count().sort("count", ascending=False).show()

+--------------------+------+
|    NAME_INCOME_TYPE| count|
+--------------------+------+
|             Working|226104|
|Commercial associate|100757|
|           Pensioner| 75493|
|       State servant| 36186|
|             Student|    17|
+--------------------+------+



# que tipo de income, educacion y genero tiene la persona que gana mas y la persona que gana la  ganancia media (o la mas cercana)

In [ ]:
# de pyspark.sql.functions puedes usar la funcion max y/o, pero tambien podrias no importar nada y usar el metodo
# .agg() no tengas miedo de probar varias formas que se te ocurran
# Puede que necesite usar el metodo .first() junto con un indexing, asi -> .first()[0]

In [ ]:
from pyspark.sql.functions import max, mean

In [ ]:
mx_value = app_record.select(max(col('AMT_INCOME_TOTAL'))).first()[0]

In [ ]:
mx_value

6750000.0

In [ ]:
app_record.select("*").where(col('AMT_INCOME_TOTAL') == float(mx_value)).show()

+-------+-----------+------------+---------------+------------+----------------+----------------+-------------------+------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+
|     ID|CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|CNT_CHILDREN|AMT_INCOME_TOTAL|NAME_INCOME_TYPE|NAME_EDUCATION_TYPE|NAME_FAMILY_STATUS|NAME_HOUSING_TYPE|DAYS_BIRTH|DAYS_EMPLOYED|FLAG_MOBIL|FLAG_WORK_PHONE|FLAG_PHONE|FLAG_EMAIL|OCCUPATION_TYPE|CNT_FAM_MEMBERS|
+-------+-----------+------------+---------------+------------+----------------+----------------+-------------------+------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+
|5987963|          M|           Y|              N|           0|       6750000.0|         Working|   Higher education|           Married|House / apartment|    -19341|         -443|         1|              1|         1|

In [ ]:
mean_value = app_record.select(mean(col('AMT_INCOME_TOTAL'))).first()[0]

In [ ]:
app_record.select("*").where((col('AMT_INCOME_TOTAL') <= float(mean_value) + 50) & (col('AMT_INCOME_TOTAL') >= float(mean_value) - 50)).show()

+-------+-----------+------------+---------------+------------+----------------+--------------------+-------------------+--------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+
|     ID|CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|CNT_CHILDREN|AMT_INCOME_TOTAL|    NAME_INCOME_TYPE|NAME_EDUCATION_TYPE|  NAME_FAMILY_STATUS|NAME_HOUSING_TYPE|DAYS_BIRTH|DAYS_EMPLOYED|FLAG_MOBIL|FLAG_WORK_PHONE|FLAG_PHONE|FLAG_EMAIL|OCCUPATION_TYPE|CNT_FAM_MEMBERS|
+-------+-----------+------------+---------------+------------+----------------+--------------------+-------------------+--------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+
|5438254|          M|           Y|              N|           0|        187497.0|Commercial associate|  Incomplete higher|Single / not married|House / apartment|     -7764|         -208|         1|   

## como es la media de ingresos por genero

In [ ]:
mean_inc_gender = app_record.groupBy("CODE_GENDER").agg(mean('AMT_INCOME_TOTAL').alias('media_ingresos'))
mean_inc_gender.show()

+-----------+------------------+
|CODE_GENDER|    media_ingresos|
+-----------+------------------+
|          F|174523.04091044018|
|          M| 214086.6388684194|
+-----------+------------------+



# outliers

una estrategia comun para conseguir outliers es ubicar aquellos puntos de la data que caigan fuera del rango ```-3 > z_score > 3```, usando esta estrategia conseguir el porcentaje de outliers de los ingresos, usar unicamente las funciones de ```pyspark.sql.functions```

Z = (valor - valor_medio) / desviacion_estandar

In [ ]:
# de pyspark.sql.functions tienes las funciones stddev y mean, usa un .withColumn() y luego tal vez un filtrado

In [ ]:
from pyspark.sql.functions import stddev, mean, col

In [ ]:
mean_inc_val = app_record.select(mean(col("AMT_INCOME_TOTAL"))).first()[0]
stdev_inc_val = app_record.select(stddev(col("AMT_INCOME_TOTAL"))).first()[0]

In [ ]:
app_record = app_record.withColumn("z_score", (col("AMT_INCOME_TOTAL") - mean_inc_val) / stdev_inc_val)

In [ ]:
cantidad_outliers = app_record.select("*").where((col("z_score") < -3) | (col("z_score") > 3)).count()

In [ ]:
cantidad_total = app_record.count()

In [ ]:
(cantidad_outliers / cantidad_total) * 100

1.203036321390378

# realiza para cada tipo de educacion en cada genero un conteo y la media de ingresos y guarda el resultado en un archivo csv

In [ ]:
app_record.columns

['ID',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'FLAG_MOBIL',
 'FLAG_WORK_PHONE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'OCCUPATION_TYPE',
 'CNT_FAM_MEMBERS',
 'z_score']

In [ ]:
# sabias que dentro del metodo groupBy se puede incluir una lista con nombres?? y el metodo .write() con el argumento header=True seguro te ayuda...
result_df = app_record.groupBy('NAME_EDUCATION_TYPE', 'CODE_GENDER').agg(
    count('*').alias('conteo'),
    mean('AMT_INCOME_TOTAL').alias('media_ingresos'))

result_df = result_df.orderBy('CODE_GENDER')

result_df.write.csv('result.csv', mode='overwrite', header=True)

result_df.show()

+--------------------+-----------+------+------------------+
| NAME_EDUCATION_TYPE|CODE_GENDER|conteo|    media_ingresos|
+--------------------+-----------+------+------------------+
|     Academic degree|          F|   251|233049.80079681275|
|   Incomplete higher|          F|  8578| 188778.5011657729|
|Secondary / secon...|          F|203078| 160690.2914929239|
|     Lower secondary|          F|  2433|128601.53637484586|
|    Higher education|          F| 80100| 209278.1039808989|
|    Higher education|          M| 37422|262128.04610389608|
|     Academic degree|          M|    61|272139.34426229505|
|   Incomplete higher|          M|  6273| 232700.6169296987|
|Secondary / secon...|          M| 98743| 195433.0775092918|
|     Lower secondary|          M|  1618| 166989.2150803461|
+--------------------+-----------+------+------------------+



# realizar un join de ambas tablas por ID y conseguir el un conteo de cada toy_status por genero

In [ ]:
# la clase asincronica y el ejercicio anterior seguro te ayudan en esta labor!
joined_df = app_record.join(credit_df, on='ID', how='inner')

In [ ]:
toystatus_count_gender = joined_df.groupBy('CODE_GENDER', 'TOY_STATUS').count()

In [ ]:
toystatus_count_gender = toystatus_count_gender.orderBy('CODE_GENDER')
toystatus_count_gender.show()

+-----------+----------+------+
|CODE_GENDER|TOY_STATUS| count|
+-----------+----------+------+
|          F|       bad|154830|
|          F|      good|103868|
|          F|   regular|260153|
|          M|   regular|127987|
|          M|      good| 52302|
|          M|       bad| 78575|
+-----------+----------+------+



In [ ]:
toystatus_count_gender.write.csv('toystatus_count_gender.csv', mode='overwrite', header=True)